In [ ]:
import torch
import torch.nn as nn

In [ ]:
import torch.nn.functional as F
from torch import Tensor
import numpy as np
from typing import Optional, Tuple

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (self.head_dim * heads == embed_size), "embed_size needs to be divided by heads"

        self.values = nn.Linear(self.embed_size, self.embed_size, bias = False)
        self.keys = nn.Linear(self.embed_size, self.embed_size, bias = False)
        self.queries = nn.Linear(self.embed_size, self.embed_size, bias = False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size, bias = False)

    def forward(self, values, keys, query, mask):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # spliting embedding into self.heads peices
        values = self.values(values).reshape(N, value_len, self.heads, self.head_dim)
        keys = self.keys(keys).reshape(N, key_len, self.heads, self.head_dim)
        queries = self.queries(query).reshape(N, query_len, self.heads, self.head_dim)

        # query shape : (N, query_len, heads, head_dim)
        # key_shape : (N, key_len, heads, head_dim)
        # energy_shape : (N, heads, query_len,  key_len)
        energy = torch.einsum("nqhd,nkhd-> nhqk", [queries, keys])

        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        attention = torch.softmax(energy / (self.embed_size ** (1/2)), dim = 3)

        out = torch.einsum("nhqk,nvhd-> nqhd", [attention, values]).reshape(N, query_len, self.heads*self.head_dim)

        out = self.fc_out(out)
        # attention shape : (N, heads, query_len, key_len)
        # value shape : (N, value_len, heads, head_dim)
        # (N, query_len, heads, head_dim)

        return out

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion*embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion*embed_size, embed_size)
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(x + forward))

        return out

In [ ]:
class Encoder(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        embed_size,
        num_layer,
        heads,
        device,
        forward_expansion,
        dropout,
        max_length,
    ):
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout = dropout,
                    forward_expansion = forward_expansion,
                )
            ]
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N, seq_length = x.shape
        position = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        out = self.dropout(self.word_embedding(x) + self.position_embedding(position))

        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm = nn.LayerNorm(embed_size)

        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, tgt_mask):
        attention = self.attention(x, x, x, tgt_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out

In [ ]:
class Decoder(nn.Module):
    def __init__(
        self,
        tgt_vocab_size,
        embed_size,
        num_layer,
        heads,
        forward_expansion,
        dropout,
        device,
        max_length,
    ):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(tgt_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)
        self.layers = nn.ModuleList(
            [
                DecoderBlock(
                    embed_size, heads, forward_expansion, dropout, device
                )
                for _ in range(num_layer)
            ]
        )
        self.fc_out = nn.Linear(embed_size, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, en_out, src_mask, tgt_mask):
        N, seq_length = x.shape
        position = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = self.dropout(self.word_embedding(x) + self.position_embedding(position))

        for layer in self.layers:
            x = layer(x, en_out, en_out, src_mask, tgt_mask)

        output = self.fc_out(x)
        return output

In [ ]:
class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        tgt_vocab_size,
        src_pad_idx,
        tgt_pad_idx,
        embed_size = 256,
        num_layer = 6,
        forward_expansion = 4,
        heads = 8,
        dropout = 0,
        device = "cpu",
        max_length  = 100,
    ):
        super(Transformer, self).__init__()

        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layer,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length,
        )

        self.decoder = Decoder(
            tgt_vocab_size,
            embed_size,
            num_layer,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length,
        )

        self.src_pad_idx = src_pad_idx
        self.tgt_pad_idx = tgt_pad_idx
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        return src_mask.to(self.device)

    def make_tgt_mask(self, tgt):
        N, tgt_len = tgt.shape
        tgt_mask = torch.tril(torch.ones(tgt_len, tgt_len)).expand(
            N, 1, tgt_len, tgt_len
        )
        return tgt_mask.to(self.device)

    def forward(self, src, tgt):
        src_mask = self.make_src_mask(src)
        tgt_mask = self.make_tgt_mask(tgt)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(tgt, enc_src, src_mask, tgt_mask)
        return out

In [ ]:
if __name__ == "__main__":
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device = "cpu"

    x = torch.tensor([[1,5,6,4,3,9,5,2,0], [1,8,7,3,4,5,6,7,2]]).to(device)

    tgt = torch.tensor([[1,7,4,3,5,9,2,0], [1,5,6,2,4,7,6,2]]).to(device)

    src_pad_idx = 0
    tgt_pad_idx = 0
    src_vocab_size = 10
    tgt_vocab_size = 10

    model = Transformer(src_vocab_size, tgt_vocab_size, src_pad_idx, tgt_pad_idx).to(device)

    out = model(x, tgt[:, :-1])
    print(out.shape)

torch.Size([2, 7, 10])


In [ ]:
!pip install torchtext

In [ ]:
from gensim.models import Word2Vec
import numpy as np

# Example sentence (5-6 lines)
# text = '''Once, there was a miser who would hide all his gold under
#           the stones in his garden. He never spent his gold, and without
#           fail, he would count his coins before going to bed every day.
#           One day a thief spotted him counting his coins. After the miser
#           went to sleep, the thief stole all the gold.'''

text = "attention is all you need is a paper written by an indian"

# Tokenization and dataset creation
tokens = text.split()
n = 3  # Using 3-grams

# Generate n-grams
ngrams = [tokens[i:i + n] for i in range(len(tokens) - n + 1)]

# Word2Vec embeddings
word2vec_model = Word2Vec([tokens], vector_size=100, window=5, min_count=1, sg=1)  # Adjust parameters as needed

# Use the Word2Vec model to get word embeddings
input_sequences = [gram[:n-1] for gram in ngrams]
next_words = [gram[-1] for gram in ngrams]

# Convert sequences and words to Word2Vec embeddings
input_vectors = [word2vec_model.wv[word] for word_sequence in input_sequences for word in word_sequence]
next_word_vectors = [word2vec_model.wv[word] for word in next_words]

# Convert to numpy arrays
input_vectors = np.array(input_vectors)
next_word_vectors = np.array(next_word_vectors)

# Now you can combine the input_vectors and next_word_vectors to train your model


In [ ]:
len(next_word_vectors), len(next_word_vectors[0])

(10, 100)

In [ ]:
len(input_vectors), len(input_vectors[0])

(20, 100)

In [ ]:
    print("Shape of input_vectors:", input_vectors.shape)
print("Shape of next_word_vectors:", next_word_vectors.shape)

Shape of input_vectors: (32, 100)
Shape of next_word_vectors: (16, 100)
